<a href="https://colab.research.google.com/github/SinSham/Email-Scraper/blob/main/EmailScraper_mt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests-html
!pip install re
!pip install request
!pip install bs4

In [29]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from collections import deque
from urllib.parse import urlsplit
import pandas as pd
from requests_html import HTMLSession
import threading
import time
from tqdm import tqdm

In [30]:
url = ["https://www.thapar.edu"]
EMAIL_REGEX = r"""(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*|"(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9]))\.){3}(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9])|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])"""

In [ ]:
def get_urls_of_xml(xml_url):
    r = requests.get(xml_url)
    xml = r.text
    soup = BeautifulSoup(xml)

    links_arr = []
    for link in soup.findAll('loc'):
        linkstr = link.getText('', True)
        links_arr.append(linkstr)

    return links_arr



links_data_arr = get_urls_of_xml("https://thapar.edu/sitemap.xml")
#print(links_data_arr)


In [32]:
session = HTMLSession()

In [35]:
email=set()
email_limit = 100
batch_size = 10
for batch_start in range(0, len(links_data_arr), batch_size):
        batch = links_data_arr[batch_start:batch_start+batch_size]
        for i in tqdm(batch):
          r = session.get(i)
          for re_match in re.finditer(EMAIL_REGEX, r.html.raw_html.decode()):
            email.add(((re_match.group())).replace("-",""))
            if len(email) >= email_limit:
              break  # Exit the inner loop if the email limit is reached

        if len(email) >= email_limit:
          break

100%|██████████| 10/10 [00:05<00:00,  1.72it/s]


In [36]:
df = pd.DataFrame(email)
df.to_csv('Emails.csv', index=False)
df

,0
0,ajaykumar@thapar.edu
1,caretaker.o@thapar.edu
2,khushneet.jindal@thapar.edu
3,faps@thapar.edu
4,caretaker.pg@thapar.edu
...,...
95,psjolly@thapar.edu
96,froshweek@thapar.edu
97,wardenb@thapar.edu
98,lav.sharma@thapar.edu
